In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("./../..")

# standard lib
import shutil
from pathlib import Path

# external imports
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as T
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm
import scipy as sp
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

# local imports
from datasets import AffNIST
from effcn.layers import SquashHinton, CapsLayer
from effcn.functions import margin_loss, masking
from misc.utils import count_parameters
from misc.plot_utils import plot_couplings, plot_capsules, plot_mat, plot_mat2
from misc.metrics import *

In [ ]:
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu"  
device = torch.device(dev)

# Data

In [ ]:
transform_train = T.Compose([
    T.RandomAffine(degrees=(-8, 8),
                   shear=(-15, 15),
                   scale=(0.9, 1.1)
                  ),
    #T.Normalize((0.0641,), (0.2257))
])
transform_valid = T.Normalize((0.0641,), (0.2257))

#p_data = '/home/matthias/projects/EfficientCN/data'
p_data = Path("C:\\Users\\matth\\projects\\EfficientCN\\data")

ds_mnist_train = AffNIST(p_root=p_data, split="mnist_train", download=True, transform=transform_train, target_transform=None)
ds_mnist_valid = AffNIST(p_root=p_data, split="mnist_valid", download=True, transform=None, target_transform=None)
ds_affnist_valid = AffNIST(p_root=p_data, split="affnist_valid", download=True, transform=None, target_transform=None)

In [ ]:
bs = 512
dl_mnist_train = torch.utils.data.DataLoader(
    ds_mnist_train, 
    batch_size=bs, 
    shuffle=True,
    pin_memory=True,
    num_workers=4)
dl_mnist_valid= torch.utils.data.DataLoader(
    ds_mnist_valid, 
    batch_size=bs, 
    shuffle=True, 
    pin_memory=True,
    num_workers=4)
dl_affnist_valid= torch.utils.data.DataLoader(
    ds_affnist_valid, 
    batch_size=bs, 
    shuffle=True, 
    pin_memory=True,
    num_workers=4)

In [ ]:
x, _ = next(iter(dl_mnist_train))
x_vis_train = x[:32]

x, _ = next(iter(dl_mnist_valid))
x_vis_mnist_valid = x[:32]

x, _ = next(iter(dl_affnist_valid))
x_vis_affnist_valid = x[:32]

In [ ]:
plt.imshow(torchvision.utils.make_grid(x_vis_train).permute(1,2,0))
plt.show()
#
plt.imshow(torchvision.utils.make_grid(x_vis_mnist_valid).permute(1,2,0))
plt.show()
#
plt.imshow(torchvision.utils.make_grid(x_vis_affnist_valid).permute(1,2,0))
plt.show()

In [ ]:
x_vis_affnist_valid.min()

# Models

### Backbone

In [ ]:
class Backbone(nn.Module):
    """
        Backbone model for AffNIST (40x40)
    """

    def __init__(self, ch_in=1, ch_out=128, n_classes=10):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(ch_in, 32, kernel_size=(5, 5), padding="valid"),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, kernel_size=(3, 3), padding="valid"),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=2, padding="valid"),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, ch_out, kernel_size=(3, 3), stride=2, padding="valid"),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(ch_out),
            nn.Conv2d(ch_out, ch_out, kernel_size=(7,7), groups=ch_out)
        )
        self.fc = nn.Linear(ch_out , n_classes)

    def forward(self, x):
        """
            IN:
                x (b, 1, 40, 40)
            OUT:
                x (b, 128, 15, 15)
        """
        x = self.layers(x)
        x = torch.flatten(x, 1)     # -> (b, 256), remove 1 X 1 grid and make vector of tensor shape 
        x = self.fc(x)
        return x

In [ ]:
# Check if it works
model = Backbone(ch_in=1, ch_out=256, n_classes=10)
model.fc = torch.nn.Identity()
y = model(torch.rand(1, 1, 40, 40))
print(count_parameters(model))
print(y.shape)

### Decoder

In [ ]:
class Decoder(nn.Module):
    """
        Decoder model for AffNIST (40x40)
        Except for last layer identical with MNIST Decoder
    """

    def __init__(self, d_in, d_out):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(d_in, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, d_out),
            nn.Sigmoid()
        )

    def forward(self, x):
        """
            IN:
                x (b, d_in) 
            OUT:
                x_rec (b, d_out)
            Notes:
                input must be masked!
        """
        x = self.layers(x)
        return x

In [ ]:
model = Decoder(d_in=32, d_out=40*40)
y=model(torch.rand(1, 32))
y.shape

In [ ]:
class AgreementRouting(nn.Module):
    def __init__(self, n_l, n_h, n_iter, sm_temp):
        super(AgreementRouting, self).__init__()
        self.n_iter = n_iter
        self.b = nn.Parameter(torch.zeros((n_l, n_h)))
        self.squash =  SquashHinton()
        self.sm_temp = sm_temp

    def forward(self, u_predict):
        v, _ = self.forward_debug(u_predict)
        return v

    def forward_debug(self, u_predict):
        batch_size, n_l, n_h, output_dim = u_predict.size()

        c = F.softmax(self.b / self.sm_temp, dim=-1)
        s = (c.unsqueeze(2) * u_predict).sum(dim=1)
        v = self.squash(s)

        if self.n_iter > 0:
            b_batch = self.b.expand((batch_size, n_l, n_h))
            for r in range(self.n_iter):
                v = v.unsqueeze(1)
                b_batch = b_batch + (u_predict * v).sum(-1)

                c = F.softmax(b_batch.view(-1, n_h) / self.sm_temp, dim=-1).view(-1, n_l, n_h, 1)
                s = (c * u_predict).sum(dim=1)
                v = self.squash(s)
        return v, c.squeeze()


class CapsLayer(nn.Module):
    def __init__(self, n_l, d_l, n_h, d_h, n_iter, sm_temp):
        super(CapsLayer, self).__init__()
        self.d_l = d_l
        self.n_l = n_l
        self.d_h = d_h
        self.n_h = n_h
        self.weights = nn.Parameter(torch.Tensor(n_l, d_l, n_h * d_h))
        self.routing_module = AgreementRouting(n_l, n_h, n_iter, sm_temp)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / np.sqrt(self.n_l)
        self.weights.data.uniform_(-stdv, stdv)

    def forward(self, caps_output):
        v, _ = self.forward_debug(caps_output)
        return v

    def forward_debug(self, caps_output):
        caps_output = caps_output.unsqueeze(2)
        u_predict = caps_output.matmul(self.weights)
        u_predict = u_predict.view(u_predict.size(0), self.n_l, self.n_h, self.d_h)
        v, c = self.routing_module.forward_debug(u_predict)
        return v, c

### CapsNet

In [ ]:
class DeepCapsNet(nn.Module):
    """
        A Deeper CN that allows
    """
    def __init__(self, d_img, ns, ds, n_iter, sm_temp):
        super().__init__()
        self.ns = ns
        self.ds = ds
        self.n_iter = n_iter
        self.d_img  = d_img
        self.sm_temp = sm_temp
        
        self.decoder = Decoder(d_in=ns[-1]*ds[-1], d_out=d_img*d_img)
        self.backbone = Backbone(ch_in=1, ch_out=ns[0] * ds[0])
        self.backbone.fc = nn.Identity()
        
        self.squash = SquashHinton()
        
        layers = []
        for idx in range(1, len(ns), 1):
            n_l = ns[idx - 1]
            n_h = ns[idx]
            d_l = ds[idx - 1]
            d_h = ds[idx]
            layers.append(CapsLayer(n_l=n_l, n_h=n_h, d_l=d_l, d_h=d_h, n_iter=n_iter, sm_temp=sm_temp))
        self.layers = nn.Sequential(*layers)


    def forward(self, x, y=None):
        b, c, h, w = x.shape
        
        x = self.backbone(x)
        
        # primecaps
        x = self.squash(x.view(-1, self.ns[0], self.ds[0]))
        
        for layer in self.layers:
            x = layer(x)
        
        # decoder
        x_rec = masking(x, y)  # also flattened
        x_rec = self.decoder(x_rec)
        x_rec = x_rec.reshape((b, c, h, w))
        return x, x_rec

    def forward_debug(self, x, y=None):
        b, c, h, w = x.shape
        
        x = self.backbone(x)
        
        # primecaps
        x = self.squash(x.view(-1, self.ns[0], self.ds[0]))
        
        us = [torch.clone(x)]
        cc = []
        # fccaps
        for layer in self.layers:
            x, cs = layer.forward_debug(x)
            cc.append(cs.detach())
            us.append(torch.clone(x).detach())
        
        # decoder
        x_rec = masking(x, y)  # also flattened
        x_rec = self.decoder(x_rec)
        x_rec = x_rec.reshape((b, c, h, w))
        return x, cc, us, x_rec

In [ ]:
def train(model, num_epochs=10):
    optimizer = optim.Adam(model.parameters(), lr = 1e-3, weight_decay=1e-5)
    lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.96)
    #
    rec_loss = torch.nn.MSELoss()
    for epoch_idx in range(num_epochs):
        # ####################
        # TRAIN
        # ####################
        model.train()
        desc = "Train [{:3}/{:3}]:".format(epoch_idx, num_epochs)
        pbar = tqdm(dl_mnist_train, bar_format=desc + '{bar:10}{r_bar}{bar:-10b}')

        for x,y_true in pbar:
            x = x.to(device)
            y_true = y_true.to(device)
            optimizer.zero_grad()

            u_h, x_rec = model.forward(x, y_true)

            # LOSS MARGIN
            y_one_hot = F.one_hot(y_true, num_classes=10)
            loss_margin = margin_loss(u_h, y_one_hot)

            # LOSS REC
            loss_rec = rec_loss(x, x_rec)

            loss = loss_margin + alpha * loss_rec
            if torch.isnan(loss):
                raise Exception("Is Nan")
            loss.backward()

            optimizer.step()

            y_pred = torch.argmax(torch.norm(u_h, dim=2), dim=1)
            acc = (y_true == y_pred).sum() / y_true.shape[0]

            pbar.set_postfix(
                    {'loss': loss.item(),
                     'acc': acc.item(),
                     'loss_rec': loss_rec.item(),
                     'loss_mar': loss_margin.item(),
                     }
            )

In [ ]:
import pickle

In [ ]:
ns = [32, 10, 10, 10]
ds = [8, 8, 8, 8, 8]
n_iter = 3
alpha = 0.005
model = DeepCapsNet(ns=ns, ds=ds, n_iter=n_iter, d_img=40, sm_temp=0.1)
model = model.to(device)
train(model, 4)

In [ ]:
alpha = 0.005
DS = [8, 16]
NS = [10, 16, 32]
DP = [2]
#
N_ITER = [1, 3, 10]
SMT = [10, 1, 0.1]
num_epochs = 2
#

working_set = []

idx = 1
max_idx = len(DS) * len(NS) * len(DP) * len(N_ITER) * len(SMT)
#
for dss in DS:
    for nss in NS:
        for dp in DP:
            for n_iter in N_ITER:
                for smt in SMT:
                    ns = [128] +  [nss]*dp + [10]
                    ds = [dss] + [dss]*dp + [dss]
                    model = DeepCapsNet(ns=ns, ds=ds, n_iter=n_iter, d_img=40, sm_temp=smt)
                    model = model.to(device)
                    print("{}/{} ds={} ns={} dp={}, n_iter={}, smt={}".format(idx, max_idx, ds, ns, dp, n_iter, smt))
                    try:
                        train(model, num_epochs)
                        working_set.append((ds, ns, dp, n_iter, smt, True))
                    except Exception as e:
                        print(e)
                        working_set.append((ds, ns, dp, n_iter, smt, False))
                    with open("results2.pkl", "wb") as file:
                        pickle.dump(working_set, file)
                    idx += 1

In [ ]:
alpha = 0.005
DS = [8, 16]
NS = [10, 16, 32, 64]
DP = [2, 3]
#
N_ITER = [1, 3, 10]
SMT = [10, 1, 0.1]
num_epochs = 2
#

working_set = []

idx = 1
max_idx = len(DS) * len(NS) * len(DP) * len(N_ITER) * len(SMT)
#
for dss in DS:
    for nss in NS:
        for dp in DP:
            for n_iter in N_ITER:
                for smt in SMT:
                    ns = [nss]*dp + [10]
                    ds = [dss]*dp + [dss]
                    model = DeepCapsNet(ns=ns, ds=ds, n_iter=n_iter, d_img=40, sm_temp=smt)
                    model = model.to(device)
                    print("{}/{} ds={} ns={} dp={}, n_iter={}, smt={}".format(idx, max_idx, ds, ns, dp, n_iter, smt))
                    try:
                        train(model, num_epochs)
                        working_set.append((ds, ns, dp, n_iter, smt, True))
                    except Exception as e:
                        print(e)
                        working_set.append((ds, ns, dp, n_iter, smt, False))
                    with open("results.pkl", "wb") as file:
                        pickle.dump(working_set, file)
                    idx += 1

In [ ]:
help(pickle.dump)

In [ ]:
max_idx = len(DS) * len(NS) * len(DP) * len(N_ITER) * len(SMT)
max_idx

# Visualize and Analyze

### Show parse tree and activations for individual samples

In [ ]:
x, y = next(iter(dl_affnist_valid))

model.eval()
with torch.no_grad():
    u_h, CC, US, x_rec = model.forward_debug(x.to(device))
y_pred = torch.argmax(torch.norm(u_h, dim=2), dim=1)
y_pred = y_pred.detach().cpu().numpy()
#
US = [u.cpu().numpy() for u in US]
CS = [c.cpu().numpy() for c in CC]
#
Y_true = y.cpu().numpy()
Y_pred = y_pred

In [ ]:
vis_class = 9
vis_max = 32
for idx in range(vis_max):
    if vis_class is not None and Y_true[idx] != vis_class:
        continue
    cs = [c[idx] for c in CS]
    us = [u[idx] for u in US]
    u_norms = [np.linalg.norm(u, axis=1) for u in us]
    
    # plot stuff
    fig, axes = plt.subplots(1, 2, figsize=(8, 4))
    title = "exp={} a={}".format(y[idx], y_pred[idx])
    #
    plot_couplings(cs, title=title, ax=axes[0], show=False)
    plot_capsules(u_norms, title=title , ax=axes[1], show=False)
    plt.show()

# Statistics For Further Evaluation and Visualization

In [ ]:
model.eval()

YY = []
CC = [[] for _ in range(len(ns) - 1)]
US = [[] for _ in range(len(ns))]


# use whole dataset
for x,y_true in dl_affnist_valid:
    x = x.to(device)
        
    with torch.no_grad():
        _, cc, us, _ = model.forward_debug(x.to(device))
        for idx in range(len(cc)):
            CC[idx].append(cc[idx].detach().cpu().numpy())
        for idx in range(len(us)):
            US[idx].append(us[idx].detach().cpu().numpy())
        YY.append(y_true.numpy())
        
# Dataset Labels
YY = np.concatenate(YY)

# Dataset Coupling Coefficient Matrices
CC = [np.concatenate(c) for c in CC]

# Dataset Capsules
US = [np.concatenate(u) for u in US]

### Mean parse tree and mean activation for dataset

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    
# Mean parse tree
cc_mean = [np.mean(c, axis=0) for c in CC]
cc_std = [np.std(c, axis=0) for c in CC]
plot_couplings(cc_mean, ax=axes[0], show=False, title="mean couplings")
plot_couplings(cc_std, ax=axes[1], show=False, title="std couplings")
    
# mean and std capsule activation
us_mean = [np.linalg.norm(u, axis=-1).mean(axis=0) for u in US]
us_std = [np.linalg.norm(u, axis=-1).std(axis=0) for u in US]
plot_capsules(us_mean, scale_factor=1, ax=axes[2], show=False, title="mean activation")
plot_capsules(us_std, scale_factor=1, ax=axes[3], show=False, title="std activation")
plt.suptitle("dataset")
plt.show()

### Parse tree from normalized Couplings

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(4 * len(CC), 4))

CNS = [normalize_couplings(C) for C in CC]

CNS_MAN = [ma_couplings(C, pr) for C, pr in CNS]
CNS_MAX = [C.max(axis=0) for C, pr in CNS]
CNS_STD = [stda_couplings(C, pr) for C, pr in CNS]

plot_couplings(CNS_MAN, ax=axes[0], show=False, title="mean")
plot_couplings(CNS_STD, ax=axes[1], show=False, title="std")
plot_couplings(CNS_MAX, ax=axes[2], show=False, title="max")
plt.show()

### Classwise mean parse tree and mean activation

In [ ]:
# mean and variance activation
for cls in range(10):
    idcs = np.where(YY == cls)[0]
    
    fig, axes = plt.subplots(1, 6, figsize=(24, 4))
    
    cc = [C[idcs] for C in CC]
    CNS = [normalize_couplings(C, eps_rate=0.5) for C in cc]
    
    CNS_MAN = [ma_couplings(C, pr) for C, pr in CNS]
    CNS_MAX = [C.max(axis=0) for C, pr in CNS]
    CNS_STD = [stda_couplings(C, pr) for C, pr in CNS]

    plot_couplings(CNS_MAN, ax=axes[0], show=False, title="mean")
    plot_couplings(CNS_STD, ax=axes[1], show=False, title="std")
    plot_couplings(CNS_MAX, ax=axes[2], show=False, title="max")
    
    # mean and std capsule activation
    us = [u[idcs] for u in US]
    us_mean = [np.linalg.norm(u, axis=-1).mean(axis=0) for u in us]
    us_std = [np.linalg.norm(u, axis=-1).std(axis=0) for u in us]
    us_max = [np.linalg.norm(u, axis=-1).max(axis=0) for u in us]
    
    plot_capsules(us_mean, scale_factor=1, ax=axes[3], show=False, title="mean activation")
    plot_capsules(us_std, scale_factor=1, ax=axes[4], show=False, title="std activation")
    plot_capsules(us_max, scale_factor=1, ax=axes[5], show=False, title="max activation")
    plt.suptitle("class {}".format(cls))
    plt.show()

# Coupling Death vs Alive

In [ ]:
c_th_mu = 1e-2
c_th_sd = 1e-2

fig, axes = plt.subplots(len(US), 2, figsize=(6, 3 * len(US)))
#
US_alive = []
for idx in range(len(US)):
    U = US[idx]
    U_norm = np.linalg.norm(U, axis=2)
    U_norm_mu = U_norm.mean(axis=0)
    U_norm_sd = U_norm.std(axis=0)
    #
    U_dead = (U_norm_sd < 1e-2) * (U_norm_mu < 1e-2)
    #
    xx = range(len(U_norm_mu))
    axes[idx][0].set_title("mu(norm(U))")
    axes[idx][0].bar(xx, U_dead, color="red",alpha=0.1)
    axes[idx][0].bar(xx, U_norm_mu)
    axes[idx][0].set_ylim(0, 1)
    axes[idx][1].set_title("sd(norm(U))")
    axes[idx][1].bar(xx, U_norm_sd)
    axes[idx][1].bar(xx, U_dead, color="red",alpha=0.1)
    axes[idx][1].set_ylim(0, 1)
    U_alive = 1 - U_dead
    US_alive.append(U_alive)
plt.show()

# Metrics

### Vibrance

In [ ]:
for U in US:
    pr = rate_dead_capsules_norm(U)
    print("#Permanently Dead: {:.3f}".format(pr.mean()))

In [ ]:
for C in CC:
    pr = rate_inactive_capsules(C)
    print("Rate inactive capsules {:.3f}".format(pr.mean()))

In [ ]:
# sanity check
for idx in range(len(CC)):
    C = CC[idx]
    U = US[idx]
    #
    rnd, rac, racnd =  get_vibrance(U, C)
    #
    print("rate alive: {:.3f} rate active {:.3f} rate active of alive {:.3f}".format(
        rnd, rac, racnd))

### Bonding

In [ ]:
for idx in range(len(CC)):
    C = CC[idx]
    b = get_bonding(C)
    print_str = "bonding strength: {:.3f}"
    print(print_str.format(b))

### Dynamics

In [ ]:
for idx in range(len(CC)):
    C = CC[idx]
    dyc = get_dynamics(C)
    #
    print("dynamics: {:.3f}".format(
          dyc))

### Correlation Capsule Activation and Max Coupling

In [ ]:
for idx in range(len(CC)):
    U = US[idx]
    C = CC[idx]
    print("corr: {:.3f}".format(activation_coupling_corr(C, U)))

### metrics for whole dataset

In [ ]:
vals = []
for idx in range(len(CC)):
    C = CC[idx]
    U = US[idx]
    #
    rnd, rac, racnd = get_vibrance(U, C)
    b = get_bonding(C)
    dyc = get_dynamics(C)
    cor = activation_coupling_corr(C, U)
    vals.append((idx,
                 rnd, rac, racnd,
                 b, dyc, cor))

In [ ]:
cols = ["layer",
        "alive rate", "active rate", "active of alive rate",
        "bonding str.", "dynamics", "cor"]
df = pd.DataFrame(data=vals, columns=cols)
df

### metrics for whole dataset, but classwise

In [ ]:

vals = []

#
for cls in range(10):
    idcs = np.where(YY == cls)[0]
    for idx in range(len(CC)):
        C = CC[idx][idcs]
        U = US[idx][idcs]
        #
        rnd, rac, racnd = get_vibrance(U, C)
        b = get_bonding(C)
        dyc = get_dynamics(C)
        cor = activation_coupling_corr(C, U)
        vals.append((cls, idx,
                     rnd, rac, racnd,
                     b, dyc, cor))

In [ ]:
cols = ["class", "layer",
        "alive rate", "active rate", "active of alive rate",
        "bonding str.", "dynamics", "cor"]
df = pd.DataFrame(data=vals, columns=cols)
#

In [ ]:
for idx in range(len(CC)):
    sdf = df[df["layer"] == idx].drop(columns=["layer"])
    print(sdf)

# Couplings Viszalizations

#### Couplings FROM DEAD Capsules

In [ ]:
for idx in range(len(CC)):
    C = CC[idx]
    Ul_alive = US_alive[idx]
    C = C[:,np.where(Ul_alive == False)[0],:]
    
    if len(C.flatten()) < 1:
        print("No dead capsules for layer {}".format(idx))
        continue
    
    C_mu = C.mean(axis=0)
    C_sd = C.std(axis=0)
    C_mx = C.max(axis=0)
    fig, axes = plt.subplots(1, 3, figsize=(33, 11))
    plot_mat2(C_mu, ax=axes[0], vmin=0, vmax=0.5)
    plot_mat2(C_sd, ax=axes[1], vmin=0, vmax=0.5)
    plot_mat2(C_mx, ax=axes[2], vmin=0, vmax=0.5)
plt.show()

#### Couplings FROM Alive Capsules

In [ ]:
for idx in range(len(CC)):
    C = CC[idx]
    Ul_alive = US_alive[idx]
    C = C[:,np.where(Ul_alive == True)[0],:]
    
    C_mu = C.mean(axis=0)
    C_sd = C.std(axis=0)
    C_mx = C.max(axis=0)
    fig, axes = plt.subplots(1, 3, figsize=(42, 14))
    plot_mat2(C_mu, ax=axes[0], vmin=0, vmax=0.5)
    plot_mat2(C_sd, ax=axes[1], vmin=0, vmax=0.5)
    plot_mat2(C_mx, ax=axes[2], vmin=0, vmax=0.5)
plt.show()

per sample count max coupling and use max to find out if coupling in general gets lower or just the average as they are loosly connected


### Couplings FROM ALIVE to DEAD

In [ ]:
for idx in range(len(CC)):
    C = CC[idx]
    Ul_alive = US_alive[idx]
    Uh_alive = US_alive[idx + 1]
    if (1 - Uh_alive).sum() < 1:
        print("{} No dead capsules for upper layer {}".format(idx ,idx + 1))
        continue
    
    
    C = C[:,np.where(Ul_alive == True)[0],:][:,:,np.where(Uh_alive == False)[0]]
    
    if len(C.flatten()) < 1:
        print("No dead capsules for layer {}".format(idx))
        continue
    
    C_mu = C.mean(axis=0)
    C_sd = C.std(axis=0)
    C_mx = C.max(axis=0)
    fig, axes = plt.subplots(1, 3, figsize=(33, 11))
    plot_mat2(C_mu, ax=axes[0], vmin=0, vmax=0.5)
    plot_mat2(C_sd, ax=axes[1], vmin=0, vmax=0.5)
    plot_mat2(C_mx, ax=axes[2], vmin=0, vmax=0.5)
plt.show()

### Couplings FROM ALIVE TO ALIVE

In [ ]:
for idx in range(len(CC)):
    C = CC[idx]
    Ul_alive = US_alive[idx]
    Uh_alive = US_alive[idx + 1]
    if (Uh_alive).sum() < 1:
        print("No dead capsules for upper layer {}".format(idx + 1))
        continue
    
    
    C = C[:,np.where(Ul_alive == True)[0],:][:,:,np.where(Uh_alive == True)[0]]
    
    if len(C.flatten()) < 1:
        print("No dead capsules for layer {}".format(idx))
        continue
    
    C_mu = C.mean(axis=0)
    C_sd = C.std(axis=0)
    C_mx = C.max(axis=0)
    fig, axes = plt.subplots(1, 3, figsize=(33, 11))
    plot_mat2(C_mu, ax=axes[0], vmin=0, vmax=0.5)
    plot_mat2(C_sd, ax=axes[1], vmin=0, vmax=0.5)
    plot_mat2(C_mx, ax=axes[2], vmin=0, vmax=0.5)
plt.show()

# CNN Only Baseline

In [ ]:
"""
epochs = 050, acc = 98,0, 73.0
epochs = 101, acc = 98.5, 74.1
epochs = 201, acc = 
"""

In [ ]:
model = CustomBB(ch_in=1, n_classes=10)
#
model = model.to(device)
#backbone
optimizer = optim.Adam(model.parameters(), lr = 1e-3, weight_decay=2e-5)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.96)
#
criterion = nn.CrossEntropyLoss()

In [ ]:
print(count_parameters(model))

In [ ]:
num_epochs = 101
#
for epoch_idx in range(num_epochs):
    # ####################
    # TRAIN
    # ####################
    model.train()
    desc = "Train [{:3}/{:3}]:".format(epoch_idx, num_epochs)
    pbar = tqdm(dl_mnist_train, bar_format=desc + '{bar:10}{r_bar}{bar:-10b}')
    
    for x,y_true in pbar:
        x = x.to(device)
        y_true = y_true.to(device)
        optimizer.zero_grad()
        
        logits = model.forward(x)
        loss = criterion(logits, y_true)

        loss.backward()
        
        optimizer.step()
        
        y_pred = torch.argmax(logits, dim=1)
        acc = (y_true == y_pred).sum() / y_true.shape[0]
        
        pbar.set_postfix(
                {'loss': loss.item(),
                 'acc': acc.item()
                 }
        )
    lr_scheduler.step()
    #
    # ####################
    # VALID
    # ####################
    if epoch_idx % 5 != 0:
        continue
    
    model.eval()
    
    total_correct = 0
    total = 0

    for x,y_true in dl_mnist_valid:
        x = x.to(device)
        y_true = y_true.to(device)
        
        with torch.no_grad():
            logits = model.forward(x)
            
            y_pred = torch.argmax(logits, dim=1)
            total_correct += (y_true == y_pred).sum()
            total += y_true.shape[0]
    print("   mnist acc_valid: {:.3f}".format(total_correct / total))

    model.eval()
    
    total_correct = 0
    total = 0

    for x,y_true in dl_affnist_valid:
        x = x.to(device)
        y_true = y_true.to(device)
        
        with torch.no_grad():
            logits = model.forward(x)
            
            y_pred = torch.argmax(logits, dim=1)
            total_correct += (y_true == y_pred).sum()
            total += y_true.shape[0]
    print("   affnist acc_valid: {:.3f}".format(total_correct / total))